In [1]:
%load_ext autoreload
%autoreload 2
#%reload_ext autoreload

In [1]:
import tensorflow as tf
import tensorflow.keras.layers as layers
import numpy as np
import matplotlib.pyplot as plt
import model_definitions
import pandas as pd
import Generator_OneCar as Generator 

### Open input file

In [2]:
SEQUENCE_LENGTH=30
ONE_CAR=True
PATH=r"C:\Users\dansp\OneDrive\Desktop\Tesi\Logs\OneCar_Pos_Vel_Rot.txt"
BATCH_SIZE=256*5
FEATURES=5 

In [3]:
maxSpeed=40
frequency=0.02
maxdiff=maxSpeed*frequency
maxAngularVelocity=140 #140 degrees for second
maxAngDiff=maxAngularVelocity*frequency
print(f"Max speed diff: {maxdiff}\nMax x/z diff: {maxdiff}\nMax rot diff: {maxAngDiff}")

Max speed diff: 0.8
Max x/z diff: 0.8
Max rot diff: 2.8000000000000003


In [4]:
Generator.SEQUENCE_LENGTH=SEQUENCE_LENGTH
Generator.FEATURES=FEATURES
df=Generator.single_care_dataframe(PATH)

In [5]:
def SubtractDF(df):
    return df.groupby("RACE",group_keys=False).apply(Generator.subtraction_columns)

dfs=[]
for i in range(len(df)):
    dfs.append(SubtractDF(df[i]))

united=dfs[0]
for i in range(1,len(df)):
    united=pd.concat([united,dfs[i] ],ignore_index=True)

In [6]:
df_train,df_val,df_test= Generator.split_train_validation_test(united,["RACE","GROUP"])

In [7]:
df_train.tail()

,DIFF_X,DIFF_Z,DIFF_VEL_X,DIFF_VEL_Z,DIFF_ROT,TIME,RACE,GROUP
7294685,-0.2441,-1.4616,-1.105577,0.44929,2.9616,0.06,448,2
7294686,-0.2908,-1.4482,-0.840877,0.14517,2.3603,0.06,448,2
7294687,-0.3010,-1.4370,0.633449,0.14549,0.2663,0.06,448,2
7294688,-0.3106,-1.4134,-0.336976,0.49492,-0.2922,0.06,448,2
7294689,-0.2937,-1.3714,0.162751,0.70257,-0.6477,0.06,448,2


In [8]:
gen_train=Generator.DataGenerator(BATCH_SIZE,df_train,16000)
gen_val=Generator.DataGenerator(BATCH_SIZE,df_val,16000)
gen_test=Generator.DataGenerator(BATCH_SIZE,df_test,16000)


Length: 7294690 races: 448 n batches: 7240930 / 1280
Length: 3647349 races: 448 n batches: 3593589 / 1280
Length: 3648017 races: 448 n batches: 3594257 / 1280


### Model definition

#### Train and print

In [10]:
if(ONE_CAR):
    model_definitions.cars=1
else:
    model_definitions.cars=4
model_definitions.features=FEATURES
model_definitions.sequence_length=SEQUENCE_LENGTH

### Rules Model

In [ ]:
lstm,history=LSTM_Train(2,SEQUENCE_LENGTH,4,rules_train,rules_val,learning_rate=0.01,epoch=100, dropout=0.2, cells=6 )

ev=lstm.evaluate(rules_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

In [ ]:
lstm2,history=LSTM_Train(2,SEQUENCE_LENGTH,4,rules_train,rules_val,learning_rate=0.002,epoch=100, dropout=0.02, cells=1)

ev=lstm2.evaluate(rules_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

In [ ]:
mlp,history=MLP_Train(2,SEQUENCE_LENGTH,4,rules_train,rules_val,learning_rate=0.001,epoch=100)

ev=mlp.evaluate(rules_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

In [ ]:
siamese,history=LSTM_Siamese_Train(2,SEQUENCE_LENGTH,4,rules_train,rules_val,learning_rate=0.001,epoch=100, dropout=0.02, cells=2 )

ev=siamese.evaluate(rules_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

### Physichs Model

Since physichs data can be very small, Each data is multiply by 10 

In [19]:
mlp_ph,history_mlp_ph=model_definitions.MLP_Train(gen_train,gen_val,learning_rate=0.0001,epoch=1,verbose=1,name="MLP",path="logs_deltaFixedUpdate")

ev=mlp_ph.evaluate(gen_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

 578/5655 [==>...........................] - ETA: 23:50 - loss: 0.9666 - mean_absolute_error: 0.6878 - accuracy: 0.0000e+00

KeyboardInterrupt: 

In [ ]:
units=32
kernel_size=1
filters=32
normalization=False
cnn_lstm,history_cnn_lstm=model_definitions.CNN_LSTM_Train(gen_train,gen_val,learning_rate=0.0005,patience=10,name=f"CNN{filters}_{kernel_size}LSTM{units}_Norm{normalization}",
                                                           path="logs_350SequenceLength",epoch=10, dropout=0.2,units=units,kernel_size=kernel_size,
                                                           filters=filters, normalization=normalization,verbose=1
                                                          )

ev=cnn_lstm.evaluate(gen_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

In [14]:
lr =1e-5
units=32
cells=1
normalization=False
name=f"LSTM{cells}_{units}_{lr}_Norm{normalization}"
lstm_ph,history_lstm_ph=model_definitions.LSTM_Train(gen_train,gen_val,learning_rate=lr,patience=6,name=name,path="logs_350SequenceLength",
                                                     epoch=5, dropout=0.2, cells=cells,units=units, normalization=normalization,verbose=1)

ev=lstm_ph.evaluate(gen_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

Epoch 1/5
5209/5209 [==============================] - 8069s 2s/step - loss: 0.7198 - mean_absolute_error: 0.5742 - accuracy: 2.9996e-08 - val_loss: 0.3600 - val_mean_absolute_error: 0.3546 - val_accuracy: 1.3242e-07 - lr: 1.0000e-05
Epoch 2/5
5209/5209 [==============================] - 8169s 2s/step - loss: 0.3302 - mean_absolute_error: 0.3494 - accuracy: 1.1998e-07 - val_loss: 0.2468 - val_mean_absolute_error: 0.2842 - val_accuracy: 1.3242e-07 - lr: 1.0000e-05
Epoch 3/5
5209/5209 [==============================] - 7956s 2s/step - loss: 0.2549 - mean_absolute_error: 0.3074 - accuracy: 1.7998e-07 - val_loss: 0.1818 - val_mean_absolute_error: 0.2505 - val_accuracy: 1.3242e-07 - lr: 1.0000e-05
Epoch 4/5
5209/5209 [==============================] - 8378s 2s/step - loss: 0.2187 - mean_absolute_error: 0.2871 - accuracy: 2.3997e-07 - val_loss: 0.1533 - val_mean_absolute_error: 0.2333 - val_accuracy: 1.3242e-07 - lr: 1.0000e-05
Epoch 5/5
5209/5209 [==============================] - 7956s 2s/

In [15]:
tf.saved_model.save(lstm_ph,f'saved_model/{name}_350SL')

INFO:tensorflow:Assets written to: saved_model/LSTM1_32_1e-05_NormFalse_350SL\assets


INFO:tensorflow:Assets written to: saved_model/LSTM1_32_1e-05_NormFalse_350SL\assets


In [ ]:
model=model_definitions.LSTM_Model(0.2, cells=3,units=32, normalization=False)
model.load_weights("saved_model\LSTM3_32_1e-05_NormFalse")

In [ ]:
x=df_test.copy()
x.drop(["TIME","RACE","GROUP"],axis=1,inplace=True)
x_ten=tf.constant(x[:SEQUENCE_LENGTH])
x_ten=tf.reshape(x_ten,(1,SEQUENCE_LENGTH,FEATURES))

In [ ]:
y_p=model.predict(x_ten)

y_p

In [ ]:
y=x.iloc[SEQUENCE_LENGTH+1]
diff= y.values-y_p
print(y)
diff

In [30]:

   #def train_step(self, data):
   #    # GradientTape is a context manager that records every operation that
   #    # you do inside. We are using it here to compute the loss so we can get
   #    # the gradients and apply them using the optimizer specified in
   #    # `compile()`.
   #    
   #    with tf.GradientTape() as tape:
   #        loss,acc = self._compute_loss(data)
   #    # Storing the gradients of the loss function with respect to the
   #    # weights/parameters.
   #    gradients = tape.gradient(loss, self.siamese_network.trainable_weights)
   #    # Applying the gradients on the model using the specified optimizer
   #    self.optimizer.apply_gradients(
   #        zip(gradients, self.siamese_network.trainable_weights)
   #    )
   #    # Let's update and return the training loss metric.
   #    self.loss_tracker.update_state(loss)
   #    self.acc_tracker.update_state((acc|True),acc)
   #    return {"loss": self.loss_tracker.result(),"acc": self.acc_tracker.result()}

   #def test_step(self, data):
   #    
   #    loss,acc = self._compute_loss(data)

   #    # Let's update and return the loss metric.
   #    self.loss_tracker.update_state(loss)
   #    self.acc_tracker.update_state((acc|True),acc)
   #    return {"loss": self.loss_tracker.result(),"acc": self.acc_tracker.result()}

   #def _compute_loss(self, data):
   #    # The output of the network is a tuple containing the distances
   #    # between the anchor and the positive example, and the anchor and
   #    # the negative example.
   #    ap_distance, an_distance = self.siamese_network(data)

   #    # Computing the Triplet Loss by subtracting both distances and
   #    # making sure we don't get a negative value.
   #    loss = ap_distance - an_distance
   #    loss = tf.maximum(loss + self.margin, 0.0)
   #    return loss, (ap_distance<an_distance)
   #@property
   #def metrics(self):
   #    # We need to list our metrics here so the `reset_states()` can be
   #    # called automatically.
   #    return [self.loss_tracker,self.acc_tracker]

In [11]:
model=model_definitions.MLP_Model(32)
model.load_weights("..\saved_model\MLP")

In [15]:
class StepModel(tf.keras.models.Model):
    """The Siamese Network model with a custom training and testing loops.

    Computes the triplet loss using the three embeddings produced by the
    Siamese Network.

    The triplet loss is defined as:
       L(A, P, N) = max(‖f(A) - f(P)‖² - ‖f(A) - f(N)‖² + margin, 0)
    """

    def __init__(self, model,sequence_lenght, steps=5):
        super().__init__()
        self.model = model
        self.steps = steps
        self.sequence_length=sequence_lenght
        self.loss_tracker = tf.keras.metrics.Mean(name="loss")
        self.acc_tracker =tf.keras.metrics.BinaryAccuracy(name ="accuracy")

    def call(self, inputs):
        output=tf.map_fn(self.stepPrediction,inputs)
        return output
    
    def stepPrediction(self,inputs):
        copy=inputs
        for index in range(self.steps):
            copy=tf.reshape(copy,[1,SEQUENCE_LENGTH,FEATURES])
            output=self.model(copy)
            toAdd=tf.reshape(output,[1,FEATURES])
            copy=tf.concat([copy[0,1:],toAdd],0)
        

        return tf.reshape(output,[FEATURES])
    
step=StepModel(model,SEQUENCE_LENGTH,steps=SEQUENCE_LENGTH)

In [17]:
model_definitions.compile_and_fit(step,gen_train,gen_val,epochs=3,learning_rate=1e-5,verbose=1,name="MLP", path=".\logs")

Epoch 1/3
   9/5655 [..............................] - ETA: 3:43:04 - loss: 0.0000e+00 - mean_absolute_error: 0.5795 - accuracy: 0.0000e+00

KeyboardInterrupt: 

In [13]:
x,y=gen_test[0]


In [18]:
x_1=x
#x_1=tf.reshape(x_1,[1,SEQUENCE_LENGTH,FEATURES])
e=step(x_1)

In [27]:
mea=tf.metrics.MeanAbsoluteError()
mea(y,e).numpy()

0.6498767

In [17]:
model(x_1)

<tf.Tensor: shape=(1, 5), dtype=float32, numpy=
array([[-0.68591005,  1.4758655 ,  0.69973326,  0.27720806,  2.031918  ]],
      dtype=float32)>

In [18]:
step.evaluate(gen_test)

RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.

In [23]:
e

<tf.Tensor: shape=(1280, 5), dtype=float32, numpy=
array([[-1.1429136 ,  1.0306978 ,  0.31283352, -0.15632942,  1.2463793 ],
       [-1.054019  ,  1.0382373 ,  0.31019306, -0.17739388,  1.2712084 ],
       [-0.9731317 ,  1.0762761 ,  0.31540555, -0.18506452,  1.2739782 ],
       ...,
       [-0.07454176,  1.4714156 ,  0.33138296, -0.02074942,  0.5002009 ],
       [-0.34701526,  1.2908127 ,  0.28812438,  0.00568229,  0.22518456],
       [-0.19572984,  1.4355267 ,  0.3026237 , -0.01330975,  0.38878202]],
      dtype=float32)>